In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_theme()

# from utils import unity

In [ ]:
prim = pd.read_csv('Datasets/kaggle-presidential/primary_results.csv')
cnty = pd.read_csv('Datasets/kaggle-presidential/county_facts.csv', dtype={'fips': str})
cnty_sub = pd.read_csv(
    'Datasets/us_census_ansi_codes/national_cousub2_utf.txt',
    skip_blank_lines=True,
    dtype=str
    )
# cnty_sub pre-cleaning command:
# sed -E 's/,([^,]*, [^,]*),/,"\1",/g'
# then re-encode as utf-8 (why sed, why???)

In [ ]:
# Add state+county fips to Census subcounty data
cnty_sub['prim_fips'] = cnty_sub['STATEFP'] + cnty_sub['COUNTYFP']

# Replace nan fips in prim with actual fips from Census
nan_regex = prim.loc[prim.fips.isna(), 'county'].drop_duplicates().str.cat(sep='|')
nan_to_fill = cnty_sub[
    cnty_sub['COUNTYNAME'].str.contains(nan_regex)
    & (cnty_sub['STATE'] == 'NH')
    ].drop_duplicates(subset=['COUNTYNAME'])
for row in nan_to_fill.itertuples():
    county = row.COUNTYNAME
    prim_cnty = county.split()[0]
    prim.loc[prim['county'] == prim_cnty, 'fips'] = row.prim_fips

# ^ this can probably be done faster using some combination of replace()
# and combine()/combine_first()
# rule of thumb for performant Python: avoid loops wherever possible

# Cast fips to string & pad with 0 to at least 5 chars
prim.fips = prim.fips.astype(int).astype(str).str.pad(5, fillchar='0')

# All counties are non-counties and have no map in cnty_sub
prim = prim[~prim.state_abbreviation.isin(['AK', 'KS'])]
# Remove all subcounties (& subcounties) that have <100 votes for a party
# Will apply a more aggressive filter on finalized county level data
prim = prim[prim.groupby(['state', 'county', 'party']).votes.transform(sum) >= 100]

# Aggregate County Subdivisions

In [ ]:
# Not named county
# city, Parish, District of Columbia, Census Area, Borough, Municipality
# cnty.loc[~cnty.state_abbreviation.isna() & ~cnty['area_name'].str.contains('County'), 'area_name'].tolist()

In [ ]:
# prim.loc[prim.fips.str.len() > 5, 'state_abbreviation'].unique()

In [ ]:
# IL is the only state with data for the county and its county subs
# will process separately: sum IL duplicates together

In [ ]:
# prim2 = prim.copy()
# # prim2.loc[prim2['state_abbreviation'] == 'CT', 'county'] = prim2['county'].replace('Andover', 'wrong_thing')
# # prim2.loc[len(prim2.index)] = ['Connecticut', 'CT', 'Fairfield', '09001', 'Democrat', 'Bernie Sanders', 1000, 0.54]

# state = 'WY'
# # Get non-county entries in prim, CT
# prim_ct_cousub = prim2.loc[
#     (prim2.fips.str.len() > 5)
#     & (prim2.state_abbreviation == state)
# ].copy()
# prim_ct_cousub['old_idx'] = prim_ct_cousub.index
# prim_ct_cousub = prim_ct_cousub.reset_index(drop=True)

# # Get unique county subdivisions in prim, CT & sort
# prim_ct_notcnty_short = prim_ct_cousub['county'].drop_duplicates().sort_values()
# ct_cnty_sub_regex = cnty_sub.loc[cnty_sub['STATE'] == state, 'COUSUBNAME'].str.split().apply(lambda x: ' '.join(x[:-1])).str.cat(sep='|')
# prim_ct_notcnty_short = prim_ct_notcnty_short[prim_ct_notcnty_short.str.contains(ct_cnty_sub_regex)]

# # Get the unique county subdivisions by their full name
# prim_ct_notcnty_regex = prim_ct_notcnty_short.str.cat(sep='|')
# cnty_sub_regex = cnty_sub.loc[cnty_sub['STATE'] == state, 'COUSUBNAME'].str.cat(sep='|')

# prim_ct_notcnty_full = cnty_sub.loc[
#     cnty_sub['COUSUBNAME'].str.contains(prim_ct_notcnty_regex)
#     & (cnty_sub['STATE'] == state),
#     'COUSUBNAME'
# ].drop_duplicates().sort_values()

# # Create a dictionary to map across
# ct_mapper = dict(zip(prim_ct_notcnty_short, prim_ct_notcnty_full))

# # Replace incomplete names with full Census names
# prim_ct_cousub['cousub'] = prim_ct_cousub['county'].replace(ct_mapper)

# # What if some non-county prim entries are missing in cnty_sub? Drop them from prim!
# not_in_cnty_sub = prim_ct_cousub[prim_ct_cousub['county'] == prim_ct_cousub['cousub']]
# prim2 = prim2.drop(index=not_in_cnty_sub['old_idx'])

# # Aggregate to county level votes from county subdivisions
# ct_cousub_agg = (
#     prim_ct_cousub
#     .merge(cnty_sub, left_on=['state_abbreviation', 'cousub'], right_on=['STATE', 'COUSUBNAME'])
#     .groupby(['STATE', 'COUNTYNAME', 'party', 'candidate', 'prim_fips'])['votes'].sum().reset_index()
#     .rename(columns={'STATE': 'state_abbreviation', 'COUNTYNAME': 'county', 'prim_fips': 'fips'})
# )
# if state == 'IL':
#     # Chicago is 99% in Cook county but also spans DuPage
#     ct_cousub_agg = ct_cousub_agg[ct_cousub_agg['county'] != 'DuPage County']

# # Check if new votes are already in prim
# err = None
# if prim2[['candidate', 'fips']].isin(ct_cousub_agg[['candidate', 'fips']].to_dict('list')).all(1).any():
#     print(f"""
#     New candidate-fips entry is already in prim!
#     State: {state}
#     """)
#     err = prim2[prim2[['candidate', 'fips']].isin(ct_cousub_agg[['candidate', 'fips']].to_dict('list')).all(1)]

In [ ]:
# # Very special case! Chicago city is primarily in Cook county but is ALSO in DuPage county!
# # https://en.wikipedia.org/wiki/Chicago
# # Will drop the DuPage county values after aggregation
# # This explains the duplicated vote counts for Cook and DuPage
# prim_ct_cousub.merge(cnty_sub, left_on=['state_abbreviation', 'cousub'], right_on=['STATE', 'COUSUBNAME'])

In [ ]:
def aggregate_cousub(prim, cnty_sub, state):
    prim2 = prim.copy()

    # Get non-county entries in prim2, CT
    prim_ct_cousub = prim2.loc[
        (prim2.fips.str.len() > 5)
        & (prim2.state_abbreviation == state)
    ].copy()
    prim_ct_cousub['old_idx'] = prim_ct_cousub.index
    prim_ct_cousub = prim_ct_cousub.reset_index(drop=True)

    # Get unique county subdivisions in prim2, CT & sort
    prim_ct_notcnty_short = prim_ct_cousub['county'].drop_duplicates().sort_values()
    ct_cnty_sub_regex = cnty_sub.loc[cnty_sub['STATE'] == state, 'COUSUBNAME'].str.split().apply(lambda x: ' '.join(x[:-1])).str.cat(sep='|')
    prim_ct_notcnty_short = prim_ct_notcnty_short[prim_ct_notcnty_short.str.contains(ct_cnty_sub_regex)]

    # Get the unique county subdivisions by their full name
    prim_ct_notcnty_regex = prim_ct_notcnty_short.str.cat(sep='|')
    cnty_sub_regex = cnty_sub.loc[cnty_sub['STATE'] == state, 'COUSUBNAME'].str.cat(sep='|')

    prim_ct_notcnty_full = cnty_sub.loc[
        cnty_sub['COUSUBNAME'].str.contains(prim_ct_notcnty_regex)
        & (cnty_sub['STATE'] == state),
        'COUSUBNAME'
    ].drop_duplicates().sort_values()

    # Create a dictionary to map across
    ct_mapper = dict(zip(prim_ct_notcnty_short, prim_ct_notcnty_full))

    # Replace incomplete names with full Census names
    prim_ct_cousub['cousub'] = prim_ct_cousub['county'].replace(ct_mapper)

    # What if some non-county prim2 entries are missing in cnty_sub? Drop them from prim2!
    not_in_cnty_sub = prim_ct_cousub[prim_ct_cousub['county'] == prim_ct_cousub['cousub']]
    prim2 = prim2.drop(index=not_in_cnty_sub['old_idx'])

    # Aggregate to county level votes from county subdivisions
    ct_cousub_agg = (
        prim_ct_cousub
        .merge(cnty_sub, left_on=['state_abbreviation', 'cousub'], right_on=['STATE', 'COUSUBNAME'])
        .groupby(['STATE', 'COUNTYNAME', 'party', 'candidate', 'prim_fips'])['votes'].sum().reset_index()
        .rename(columns={'STATE': 'state_abbreviation', 'COUNTYNAME': 'county', 'prim_fips': 'fips'})
    )
    if state == 'IL':
        # Chicago is 99% in Cook county but also spans DuPage
        ct_cousub_agg = ct_cousub_agg[ct_cousub_agg['county'] != 'DuPage County']

    # Check if new county is already in prim2, WIP can check via fips
    if prim2['fips'].isin(ct_cousub_agg['fips']).any():
        print(f"""
        New county is already in prim!
        Print {state}
        Print county here
        Print fips here
        """)
        return prim2[prim2['fips'].isin(ct_cousub_agg['fips'])]

    res_us_state_abbrev = {
        'AL': 'Alabama',
        'AK': 'Alaska',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'AR': 'Arkansas',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'DC': 'District of Columbia',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'IA': 'Iowa',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'ME': 'Maine',
        'MD': 'Maryland',
        'MA': 'Massachusetts',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MS': 'Mississippi',
        'MO': 'Missouri',
        'MT': 'Montana',
        'NE': 'Nebraska',
        'NV': 'Nevada',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NY': 'New York',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'MP': 'Northern Mariana Islands',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VT': 'Vermont',
        'VI': 'Virgin Islands',
        'VA': 'Virginia',
        'WA': 'Washington',
        'WV': 'West Virginia',
        'WI': 'Wisconsin',
        'WY': 'Wyoming'
    }

    # Add state and fraction_votes to ct_cousub_agg
    ct_cousub_agg['state'] = ct_cousub_agg['state_abbreviation'].replace(res_us_state_abbrev)
    # add fraction_votes to ct_cousub_agg
    # Get total votes
    ct_cousub_agg['total_votes'] = (ct_cousub_agg
    .groupby(['county', 'state','party'])
    .votes
    .transform(sum)
    )

    ct_cousub_agg['fraction_votes'] = ct_cousub_agg['votes'] / ct_cousub_agg['total_votes']
    ct_cousub_agg = ct_cousub_agg.drop(columns=["total_votes"])
    ct_cousub_agg['county'] = ct_cousub_agg.county.str.rsplit(' ', 1, expand=True).iloc[:, 0]

    # Add them to their respective counties in prim2
    # For new counties (fips id wasn't present in prim2), simply append to prim2
    # and remove the county subdivision entries
    # ct_cousub_agg = ct_cousub_agg[prim2.columns]
    prim2 = prim2.append(ct_cousub_agg, ignore_index=True)
    prim2 = prim2[~(prim2.county.isin(prim_ct_notcnty_short) & (prim2.state_abbreviation == state))]
    return prim2

In [ ]:
# Aggregate county subdivisions to county level
states = prim.loc[prim.fips.str.len() > 5, 'state_abbreviation'].unique()
for state in states:
    prim = aggregate_cousub(prim, cnty_sub, state)

In [ ]:
# Drop rows where state/county does not match fips
# These are nonidentifiable rows
# e.g. CT, Cheshire, 33005 -> is it Cheshire, New Haven County, CT
# Or is it Cheshire County, New Hampshire?
prim['STATEFP'] = prim.fips.str[:2]
mapper = cnty_sub[['STATE', 'STATEFP']].drop_duplicates().set_index('STATEFP')['STATE'].to_dict()
prim['STATE'] = prim['STATEFP'].replace(mapper)
prim = prim[prim['state_abbreviation'] == prim['STATE']]
prim.drop(columns=['STATE', 'STATEFP'], inplace=True)

In [ ]:
# Only keep county votes with >= 1000 votes per party
prim = prim[prim.groupby(['state', 'county', 'party'])['votes'].transform(sum) >= 1000]

# County Level Data

In [ ]:
# Get unique area names where state_abbreviation is NA
# 1. Get rows where state_abbreviation is NA
# 2. Get only the area_name column
# 3. Get unique values
not_counties = cnty.loc[cnty.state_abbreviation.isna(), 'area_name'].unique()

# income = INC110213, hispanic = RHI725214, white= RHI825214, college = EDU685213, density = POP060210
# VET605213,"Veterans, 2009-2013" # vets
# POP645213,"Foreign born persons, percent, 2009-2013" # foreign
# RHI425214,"Asian alone, percent, 2014" # asian
# RHI225214,"Black or African American alone, percent, 2014" # black
# SEX255214,"Female persons, percent, 2014" # female
# AGE775214,"Persons 65 years and over, percent, 2014" # senior
# AGE295214,"Persons under 18 years, percent, 2014" # children
# county = area_name
# st_cnty = state_abbreviation + county


fts = ['PST045214','INC110213', 'RHI725214', 'RHI425214', 'RHI225214', 'RHI825214', 'POP645213', 'EDU685213', 'POP060210',
 'VET605213', 'SEX255214', 'AGE775214', 'AGE295214']
vals = ['population', 'income', 'hispanic', 'asian', 'black', 'white', 'foreign', 'college', 'density', 'vets',
 'female', 'senior', 'children']

cnty_fts = cnty[["area_name", "state_abbreviation", 'fips'] + fts].copy()
cnty_fts.rename(columns=dict(zip(fts, vals)), inplace=True)
cnty_fts.rename(columns={"area_name": 'county'}, inplace=True)
cnty_fts = cnty_fts[~cnty_fts.county.isin(not_counties)]
cnty_fts.county = cnty_fts.county.str[:-7]
cnty_fts['st_cnty'] = cnty_fts.state_abbreviation + '_' + cnty_fts.county
cnty_fts.drop(columns=['county'], inplace=True)
cnty_fts.rename(columns={'state_abbreviation': 'st_abbrev'}, inplace=True)

### Republican

In [ ]:
rep = prim.loc[prim.party == 'Republican'].copy()

# Create new column "state_county" that is concatenation of state abbreviation, _, and county
# Remove state_abbreviation, county, and fips

rep['st_cnty'] = rep.state_abbreviation + '_' + rep.county
rep.drop(columns=['county', 'fips', 'state_abbreviation'], inplace=True)

# Get total votes
rep['total_votes'] = (rep
.groupby('st_cnty')
.votes
.transform(sum)
)

# Compute unity (WIP)
# rep['unity'] = (rep
# .groupby('st_cnty')
# .fraction_votes
# .transform(unity, n=11) # 11 Democratic candidates
# )

# Pick out winners, votes, fraction_votes, and state_county from dem
# Add these stats to rep_cnty_fts via .merge()

win_df_rep = rep.loc[(rep.groupby('st_cnty').votes.transform(max) == rep.votes), rep.columns[rep.columns != 'party']]
win_df_rep = win_df_rep.loc[win_df_rep.votes > 0]
win_df_rep.rename(columns= {'candidate': "winner"}, inplace=True)
rep_cnty_fts = cnty_fts.merge(win_df_rep, on = 'st_cnty')
# Compute voter turnout %
rep_cnty_fts['voter_turnout'] = rep_cnty_fts['total_votes'] / (rep_cnty_fts['population']-(rep_cnty_fts['children']/100*rep_cnty_fts['population']))
# Filter counties that have poor turnout (< 5%)
rep_cnty_fts = rep_cnty_fts[rep_cnty_fts.voter_turnout >= 0.05]

### Democratic

In [ ]:
dem = prim.loc[prim.party == 'Democrat'].copy()

# Create new column "state_county" that is concatenation of state abbreviation, _, and county
# Remove state_abbreviation, county, and fips

dem['st_cnty'] = dem.state_abbreviation + '_' + dem.county
dem.drop(columns=['county', 'fips', 'state_abbreviation'], inplace=True)

# Get total votes
dem['total_votes'] = (dem
.groupby('st_cnty')
.votes
.transform(sum)
)

# Compute unity (WIP)
# dem['unity'] = (dem
# .groupby('st_cnty')
# .fraction_votes
# .transform(unity, n=4) # 4 Democratic candidates
# )

# Pick out winners, votes, fraction_votes, and state_county from dem
# Add these stats to dem_cnty_fts via .merge()

win_df_dem = dem.loc[(dem.groupby('st_cnty').votes.transform(max) == dem.votes), dem.columns[dem.columns != 'party']]
win_df_dem = win_df_dem.loc[win_df_dem.votes > 0]
win_df_dem.rename(columns= {'candidate': "winner"}, inplace=True)
dem_cnty_fts = cnty_fts.merge(win_df_dem, on = 'st_cnty')
# Compute voter turnout %
dem_cnty_fts['voter_turnout'] = dem_cnty_fts['total_votes'] / (dem_cnty_fts['population']-(dem_cnty_fts['children']/100*dem_cnty_fts['population']))
# Filter counties that have poor turnout (< 5%)
# dem_cnty_fts = dem_cnty_fts[dem_cnty_fts.voter_turnout >= 0.05]

### Save

In [ ]:
rep_cnty_fts.to_csv('Datasets/kaggle-presidential/rep_clean_county_facts.csv', index=False)
dem_cnty_fts.to_csv('Datasets/kaggle-presidential/dem_clean_county_facts.csv', index=False)

# State Level Data

* Get Republican and Democratic winners for each state
* Get voting data for each state (e.g. fraction of votes, etc.)
* Get demographic data for each state

In [ ]:
prim_st = prim.groupby(['state', 'party', 'candidate'])['votes'].sum().reset_index()
prim_st['total_votes'] = prim_st.groupby(['state', 'party'])['votes'].transform(sum)
prim_st['fraction_votes'] = prim_st.votes / prim_st.total_votes

In [ ]:
prim_st.head()

,state,party,candidate,votes,total_votes,fraction_votes
0,Alabama,Democrat,Bernie Sanders,74469,378840,0.196571
1,Alabama,Democrat,Hillary Clinton,304371,378840,0.803429
2,Alabama,Republican,Ben Carson,87143,833251,0.104582
3,Alabama,Republican,Donald Trump,369251,833251,0.443145
4,Alabama,Republican,John Kasich,37831,833251,0.045402


In [ ]:
# Get unique area names where state_abbreviation is NA
# 1. Get rows where state_abbreviation is NA
# 2. Get only the area_name column
# 3. Get unique values
states = cnty.loc[
    cnty.state_abbreviation.isna()
    & (cnty.area_name != 'United States'),
    'area_name'].unique()

# income = INC110213, hispanic = RHI725214, white= RHI825214, college = EDU685213, density = POP060210
# VET605213,"Veterans, 2009-2013" # vets
# POP645213,"Foreign born persons, percent, 2009-2013" # foreign
# RHI425214,"Asian alone, percent, 2014" # asian
# RHI225214,"Black or African American alone, percent, 2014" # black
# SEX255214,"Female persons, percent, 2014" # female
# AGE775214,"Persons 65 years and over, percent, 2014" # senior
# AGE295214,"Persons under 18 years, percent, 2014" # children
# county = area_name
# st_cnty = state_abbreviation + county

fts = ['PST045214','INC110213', 'RHI725214', 'RHI425214', 'RHI225214', 'RHI825214', 'POP645213', 'EDU685213', 'POP060210',
 'VET605213', 'SEX255214', 'AGE775214', 'AGE295214']
vals = ['population', 'income', 'hispanic', 'asian', 'black', 'white', 'foreign', 'college', 'density', 'vets',
 'female', 'senior', 'children']

st_fts = cnty[["area_name", "state_abbreviation", 'fips'] + fts].copy()
st_fts.rename(columns=dict(zip(fts, vals)), inplace=True)
st_fts.rename(columns={"area_name": 'state'}, inplace=True)
st_fts = st_fts[st_fts.state.isin(states)]
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
st_fts['state_abbreviation'] = st_fts.state.replace(us_state_abbrev)
st_fts.rename(columns={'state_abbreviation': 'st_abbrev'}, inplace=True)

In [ ]:
st_fts.head()

,state,st_abbrev,fips,population,income,hispanic,asian,black,white,foreign,college,density,vets,female,senior,children
1,Alabama,AL,01000,4849377,43253,4.1,1.3,26.7,66.2,3.5,22.6,94.4,388865,51.5,15.3,22.8
69,Alaska,AK,02000,736732,70760,6.8,6.1,3.9,61.9,7.0,27.5,1.2,71004,47.4,9.4,25.3
99,Arizona,AZ,04000,6731484,49774,30.5,3.3,4.7,56.2,13.4,26.9,56.3,522382,50.3,15.9,24.1
115,Arkansas,AR,05000,2966369,40768,7.0,1.5,15.6,73.4,4.5,20.1,56.0,237311,50.9,15.7,23.8
191,California,CA,06000,38802500,61094,38.6,14.4,6.5,38.5,27.0,30.7,239.1,1893539,50.3,12.9,23.6


### Republican

In [ ]:
rep_st = prim_st.loc[prim_st.party == 'Republican'].copy()

# Pick out winners, votes, fraction_votes, and state from rep
# Add these stats to rep_st_fts via .merge()

win_df_rep_st = rep_st.loc[(rep_st.groupby('state').votes.transform(max) == rep_st.votes), rep_st.columns[rep_st.columns != 'party']]
win_df_rep_st = win_df_rep_st.loc[win_df_rep_st.votes > 0]
win_df_rep_st.rename(columns= {'candidate': "winner"}, inplace=True)
rep_st_fts = st_fts.merge(win_df_rep_st, on = 'state')
# Compute voter turnout %
rep_st_fts['voter_turnout'] = rep_st_fts['total_votes'] / (rep_st_fts['population']-(rep_st_fts['children']/100*rep_st_fts['population']))
# Filter states that have poor turnout (< 5%)
rep_st_fts = rep_st_fts[rep_st_fts.voter_turnout >= 0.05]

In [ ]:
rep_st_fts.head()

,state,st_abbrev,fips,population,income,hispanic,asian,black,white,foreign,...,density,vets,female,senior,children,winner,votes,total_votes,fraction_votes,voter_turnout
0,Alabama,AL,01000,4849377,43253,4.1,1.3,26.7,66.2,3.5,...,94.4,388865,51.5,15.3,22.8,Donald Trump,369251,833251,0.443145,0.222573
1,Arizona,AZ,04000,6731484,49774,30.5,3.3,4.7,56.2,13.4,...,56.3,522382,50.3,15.9,24.1,Donald Trump,249634,434560,0.574452,0.085054
2,Arkansas,AR,05000,2966369,40768,7.0,1.5,15.6,73.4,4.5,...,56.0,237311,50.9,15.7,23.8,Donald Trump,130239,390263,0.333721,0.172654
3,California,CA,06000,38802500,61094,38.6,14.4,6.5,38.5,27.0,...,239.1,1893539,50.3,12.9,23.6,Donald Trump,1174210,1494865,0.785496,0.050425
4,Connecticut,CT,09000,3596677,69461,15.0,4.5,11.5,68.8,13.6,...,738.1,217947,51.2,15.5,21.6,Donald Trump,123315,208722,0.590810,0.074020


### Democrat

In [ ]:
dem_st = prim_st.loc[prim_st.party == 'Democrat'].copy()

# Pick out winners, votes, fraction_votes, and state_county from dem_st
# Add these stats to dem_st_fts via .merge()

win_df_dem_st = dem_st.loc[(dem_st.groupby('state').votes.transform(max) == dem_st.votes), dem_st.columns[dem_st.columns != 'party']]
win_df_dem_st = win_df_dem_st.loc[win_df_dem_st.votes > 0]
win_df_dem_st.rename(columns= {'candidate': "winner"}, inplace=True)
dem_st_fts = st_fts.merge(win_df_dem_st, on = 'state')
# Compute voter turnout %
dem_st_fts['voter_turnout'] = dem_st_fts['total_votes'] / (dem_st_fts['population']-(dem_st_fts['children']/100*dem_st_fts['population']))
# Filter counties that have poor turnout (< 5%)
dem_st_fts = dem_st_fts[dem_st_fts.voter_turnout >= 0.05]

In [ ]:
dem_st_fts.head()

,state,st_abbrev,fips,population,income,hispanic,asian,black,white,foreign,...,density,vets,female,senior,children,winner,votes,total_votes,fraction_votes,voter_turnout
0,Alabama,AL,01000,4849377,43253,4.1,1.3,26.7,66.2,3.5,...,94.4,388865,51.5,15.3,22.8,Hillary Clinton,304371,378840,0.803429,0.101193
1,Arizona,AZ,04000,6731484,49774,30.5,3.3,4.7,56.2,13.4,...,56.3,522382,50.3,15.9,24.1,Hillary Clinton,234891,397707,0.590613,0.077841
2,Arkansas,AR,05000,2966369,40768,7.0,1.5,15.6,73.4,4.5,...,56.0,237311,50.9,15.7,23.8,Hillary Clinton,131939,191231,0.689946,0.084602
3,California,CA,06000,38802500,61094,38.6,14.4,6.5,38.5,27.0,...,239.1,1893539,50.3,12.9,23.6,Hillary Clinton,1940017,3441376,0.563733,0.116086
5,Connecticut,CT,09000,3596677,69461,15.0,4.5,11.5,68.8,13.6,...,738.1,217947,51.2,15.5,21.6,Hillary Clinton,170050,322404,0.527444,0.114336


### Save

In [ ]:
rep_st_fts.to_csv('Datasets/kaggle-presidential/rep_clean_state_facts.csv', index=False)
dem_st_fts.to_csv('Datasets/kaggle-presidential/dem_clean_state_facts.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6db671ff-4b94-4ec9-9d8f-30a849bb0caf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>